# Init

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/wliao0504/code/clif/pyCLIF'

## Reload

In [2]:
import pandas as pd
from pathlib import Path
import duckdb

# Import individual table classes
from src.pyclif.tables.vitals import Vitals
from src.pyclif.tables.labs import Labs
from src.pyclif.tables.medication_admin_continuous import MedicationAdminContinuous

from importlib import reload

import yaml

with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
from pyclif.utils import sofa
reload(sofa)

<module 'pyclif.utils.sofa' from '/Users/wliao0504/code/clif/pyCLIF/src/pyclif/utils/sofa.py'>

In [3]:
# Initialize the tables
vitals = Vitals.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)

vitals_df = vitals.df

# Initialize the tables
labs = Labs.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)

labs_df = labs.df

# Initialize the tables
mac = MedicationAdminContinuous.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)

mac_df = mac.df

Loading clif_vitals.parquet
Data loaded successfully from clif_vitals.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
recorded_dttm: null count after conversion= 0
Validation completed with 1 error(s). See `errors` attribute.
Loading clif_labs.parquet
Data loaded successfully from clif_labs.parquet
lab_order_dttm: null count before conversion= 43419
lab_order_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
lab_order_dttm: null count after conversion= 43419
lab_collect_dttm: null count before conversion= 0
lab_collect_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
lab_collect_dttm: null count after conversion= 0
lab_result_dttm: null count before conversion= 0
lab_result_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
lab_result_dttm: null count after conversion= 0
Validation completed with 24 error(s). See `errors` attribute.
Loading clif_medi

In [4]:
# Mock DataFrame creation for ids_w_dttm
ids_w_dttm = pd.DataFrame({
    'hospitalization_id': ['23559586', '20626031'],
    'start_dttm': pd.to_datetime(['2132-12-15 14:29:00+00:00', '2132-12-14 21:00:00+00:00']),
    'end_dttm': pd.to_datetime(['2137-08-25 14:00:00+00:00', '2137-09-02 09:00:00+00:00'])
})

In [5]:
query_dict = {'weight_kg': ['latest', 'max'], 'spo2': ['max', 'min']}

# pivoted = sofa.lookup_extremal_vital_values(ids_w_dttm, query_dict, vitals_df, )

### Generic

In [6]:
df = sofa.lookup_extremal_values_in_long_table(ids_w_dttm, query_dict, vitals_df, "vitals")

df

,hospitalization_id,start_dttm,end_dttm,spo2_max,spo2_min,weight_kg_latest,weight_kg_max
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,100.0,91.0,95.1,95.1
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,100.0,87.0,123.6,124.5


In [7]:
query_dict = {"propofol": ["latest", "max"], "fentanyl": ["latest", "min"]}

sofa.lookup_extremal_values_in_long_table(ids_w_dttm, query_dict, mac_df, "medication_admin_continuous")

,hospitalization_id,start_dttm,end_dttm,fentanyl_latest,fentanyl_min,propofol_latest,propofol_max,propofol_min
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,NaN,NaN,0.0,50.229695,0.0
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,0.0,0.0,0.0,30.026425,0.0


In [8]:
query_dict = {
    'hemoglobin': ['latest', 'max'], 
    'platelet_count': ['max', 'min'],
    'bicarbonate': ['latest', 'min']
    }

sofa.lookup_extremal_values_in_long_table(ids_w_dttm, query_dict, labs_df, "labs")

,hospitalization_id,start_dttm,end_dttm,bicarbonate_latest,bicarbonate_min,hemoglobin_latest,hemoglobin_max,platelet_count_latest,platelet_count_max,platelet_count_min
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,27.0,22.0,7.9,9.8,NaN,375.0,192.0
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,21.0,18.0,7.4,9.3,61.0,115.0,61.0


## New

In [9]:
melted = qualified.melt(
    id_vars=['hospitalization_id', 'start_dttm', 'end_dttm', 'vital_category', 'vital_value'],
    value_vars=['rn_max', 'rn_latest', 'rn_min'],
    var_name='extreme',
    value_name='rn'
).query('rn == 1').drop(columns=['rn'])

NameError: name 'qualified' is not defined

In [ ]:
# Create column names like 'spo2_max', 'weight_kg_latest'
melted['column_name'] = melted['vital_category'] + '_' + melted['extreme'].str.replace('rn_', '')

# Pivot to get the desired column structure
pivoted = melted.pivot_table(
    index=['hospitalization_id', 'start_dttm', 'end_dttm'],
    columns='column_name',
    values='vital_value'
).reset_index()

In [ ]:
pivoted

,hospitalization_id,start_dttm,end_dttm,spo2_max,spo2_min,weight_kg_latest,weight_kg_max
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,100.0,91.0,95.1,95.1
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,100.0,87.0,123.6,124.5
